# Ajustes previos

In [663]:
# load datasets (students Portuguese scores)

import pandas as pd

d = pd.read_csv('Data/df_regresion.csv')
d.head(4)

,Identificador de persona,Identificador de hogar,Area,Sexo,Edad,Seguro Social - Alternativa 1,Seguro Social - Alternativa 2,Estado civil,Nivel de instrucción,Nivel de instrucción.1,...,Parientes,No parientes,Empleado(a) doméstico(a),Tipology,PARROQUIA_CODIGO,Número de mujeres en el hogar,Número de personas del hogar,Elección política,Votos proporcionales,random
0,10015290090200507101,100152900902005071,Rural,Mujer,74.0,"IESS, seguro general",Ninguno,Separado(a),Primaria,Educación Básica,...,0,0,0,Hogar tipo 1,100152,1,1,1030,3,5
1,10015290090200507101,100152900902005071,Rural,Mujer,74.0,"IESS, seguro general",Ninguno,Separado(a),Primaria,Educación Básica,...,0,0,0,Hogar tipo 1,100152,1,1,18,1,6
2,10015290090200507101,100152900902005071,Rural,Mujer,74.0,"IESS, seguro general",Ninguno,Separado(a),Primaria,Educación Básica,...,0,0,0,Hogar tipo 1,100152,1,1,2,2,12
3,10015290090200507101,100152900902005071,Rural,Mujer,74.0,"IESS, seguro general",Ninguno,Separado(a),Primaria,Educación Básica,...,0,0,0,Hogar tipo 1,100152,1,1,Ausentes,3,9


In [664]:
# Vamos a identificar las columnas para adecuador su tipo (str, float o int)

d.columns

Index(['Identificador de persona', 'Identificador de hogar', 'Area', 'Sexo',
       'Edad', 'Seguro Social - Alternativa 1',
       'Seguro Social - Alternativa 2', 'Estado civil', 'Nivel de instrucción',
       'Nivel de instrucción.1', 'Cómo se considera',
       'Condición de inactividad', 'Categoría de ocupación',
       'Cuántos años trabaja', 'Condición de actividad', 'Ingreso Laboral',
       'Grupo de Ocupación CIUO8 (población ocupada de 15 años y más)',
       'Rama de Actividad CIIU4 (población ocupada de 15 años y más)',
       'Ingreso per cápita', 'Pobreza', 'Extrema Pobreza',
       'Forma de tenencia de la vivienda',
       'Con relación al mes anterior, la situación económica de su hogar es:',
       'Cómo cree usted que será la situación económica de su hogar dentro de los próximos 3 meses:',
       'Planea usted o algún miembro del hogar adquirir un vehículo en los próximos 3 meses',
       'Planea usted o algún miembro del hogar adquirir un inmueble (casa, departame

In [665]:
# Depuramos variables que ya no son útiles para este análisis de árbol de decisión

d = d.drop(columns=['Nivel de instrucción.1', 
                    'Identificador de persona', 
                    'Identificador de hogar', 
                    'Ingreso Laboral',
                    'random', 
                    'Grupo de Ocupación CIUO8 (población ocupada de 15 años y más)',
                    'PARROQUIA_CODIGO',
                    'Votos proporcionales'
                   ]
          )

In [666]:
# Creamos un diccionario para modificar el tipo de datos según deseemos

NEW = {'Area': str,
       'Sexo': str,
       'Edad': float,
       'Seguro Social - Alternativa 1': str,
       'Seguro Social - Alternativa 2': str,
       'Estado civil': str,
       'Nivel de instrucción': str,
       'Cómo se considera': str,
       'Condición de inactividad': str,
       'Categoría de ocupación': str,
       'Cuántos años trabaja': float,
       'Condición de actividad': str,
       'Rama de Actividad CIIU4 (población ocupada de 15 años y más)': str,
       'Ingreso per cápita': float,
       'Pobreza': str,
       'Extrema Pobreza': str,
       'Forma de tenencia de la vivienda': str,
       'Con relación al mes anterior, la situación económica de su hogar es:': str,
       'Cómo cree usted que será la situación económica de su hogar dentro de los próximos 3 meses:': str,
       'Planea usted o algún miembro del hogar adquirir un vehículo en los próximos 3 meses': str,
       'Planea usted o algún miembro del hogar adquirir un inmueble (casa, departamento, pieza) en los próximos 3 meses': str,
       'Usted o algún miembro del hogar tiene planes de endeudamiento en los próximos 3 meses (bancos, financieras, etc.)': str,
       'Con relación al mes anterior, la situación económica del país es:': str,
       'Usted considera que durante los próximos 3 meses la situación económica del país será:': str,
       'En los 3 meses anteriores, considera que la situación en Ecuador para encontrar/mejorar un puesto de trabajo es:': str,
       'Considera usted que la situación en Ecuador para encontrar/mejorar un puesto de trabajo dentro de 3 meses será:': str,
       'Género del jefe/a de hogar': str,
       'Cónyuge': int,
       'Hijo o Hija': int,
       'Parientes': int,
       'No parientes': int,
       'Empleado(a) doméstico(a)': int,
       'Tipology': str,
       'Número de mujeres en el hogar': int,
       'Número de personas del hogar': int, 
       'Elección política': str
      }

d = d.astype(NEW)

Vamos a simplificar el modelo clasificando la elección política en 2: asignando 1 para los que votaron por UNES y 2 para los que no. 

In [667]:
# El código para la op 1030 indica si se voto por FUERZA COMPROMISO SOCIAL 

d['Elección política'] = d['Elección política'].apply(lambda election: 1 if election == '1030' else 0)

Vamos a ajustar la base, para evitar los NaNs, pues es imposible trabajar con ellos.

Detalle de ajustes:

Vamos a reemplazarlos valores perdidos como sigue:

1. Condensar las filas *Condición de inactividad* y *Condición de ocupación* en una sola fila llamada *Condición de ocupación*. Una vez hecho esto, los NaNs que aún sobren serán catalogados como *Otro motivo*.
2. Vamos a rellenar los NaNs de la variable categoría de ocupación con *No declara*.
3. Vamos a rellenar los NaNs o blancos de la variable cuántos años trabaja con 0.
4. Vamos a crear una solo variable para las variables *Pobreza* Y *Extrema pobreza* llamada *Tipo de pobreza*.
5. El *Grupo de Ocupación CIUO8 (población ocupada de 15 años y más)* y *Rama de Actividad CIIU4 (población ocupada de 15 años y más* serán depuradas con un reemplzados de NaNs por *Es inactivo*.
6. Los valores de *Ingreso per cápita* perdidos se van a excluir.


In [668]:
# Hago que nan sea reconocido como ninguno

d = d.replace('nan', None)

# Reemplazo valores de la variable Condición de ocupación con Condición de inactividad

d['Condición de actividad'].fillna(d['Condición de inactividad'], inplace=True)
d['Condición de actividad'].fillna('Otro motivo', inplace=True)
# Elimino la variable que ya no será útil

del d['Condición de inactividad']

In [669]:
# El resto de modificaciones:

d = d.replace('nan', None)

d['Categoría de ocupación'].fillna('No declara', inplace=True)
d['Cuántos años trabaja'].fillna(0, inplace=True)

# Creamos una sola variable que se llame tipo de pobreza 

d['Pobreza'].fillna(d['Extrema Pobreza'], inplace=True)
d['Pobreza'].fillna('Ninguna', inplace=True) # Lleno los que aún no han sido clasficados

# Elimino la variable que ya no será útil

del d['Extrema Pobreza']

# y renombro la variable

d.rename(columns={'Pobreza': 'Tipo de pobreza'})

d['Rama de Actividad CIIU4 (población ocupada de 15 años y más)'].fillna('Es inactivo', inplace=True)


In [670]:
d = d.loc[( (d['Ingreso per cápita'] != 0) & (d['Ingreso per cápita'] != None))]
d.columns

Index(['Area', 'Sexo', 'Edad', 'Seguro Social - Alternativa 1',
       'Seguro Social - Alternativa 2', 'Estado civil', 'Nivel de instrucción',
       'Cómo se considera', 'Categoría de ocupación', 'Cuántos años trabaja',
       'Condición de actividad',
       'Rama de Actividad CIIU4 (población ocupada de 15 años y más)',
       'Ingreso per cápita', 'Pobreza', 'Forma de tenencia de la vivienda',
       'Con relación al mes anterior, la situación económica de su hogar es:',
       'Cómo cree usted que será la situación económica de su hogar dentro de los próximos 3 meses:',
       'Planea usted o algún miembro del hogar adquirir un vehículo en los próximos 3 meses',
       'Planea usted o algún miembro del hogar adquirir un inmueble (casa, departamento, pieza) en los próximos 3 meses',
       'Usted o algún miembro del hogar tiene planes de endeudamiento en los próximos 3 meses (bancos, financieras, etc.)',
       'Con relación al mes anterior, la situación económica del país es:',
 

## Un ajuste inadvertido, pero costoso

Quiero que el lector sea consciente de que si no aproxima los decimales a 2 cifras, la librería no podrá hacer la simulación.

In [671]:
dict(d.dtypes)

{'Area': dtype('O'),
 'Sexo': dtype('O'),
 'Edad': dtype('float64'),
 'Seguro Social - Alternativa 1': dtype('O'),
 'Seguro Social - Alternativa 2': dtype('O'),
 'Estado civil': dtype('O'),
 'Nivel de instrucción': dtype('O'),
 'Cómo se considera': dtype('O'),
 'Categoría de ocupación': dtype('O'),
 'Cuántos años trabaja': dtype('float64'),
 'Condición de actividad': dtype('O'),
 'Rama de Actividad CIIU4 (población ocupada de 15 años y más)': dtype('O'),
 'Ingreso per cápita': dtype('float64'),
 'Pobreza': dtype('O'),
 'Forma de tenencia de la vivienda': dtype('O'),
 'Con relación al mes anterior, la situación económica de su hogar es:': dtype('O'),
 'Cómo cree usted que será la situación económica de su hogar dentro de los próximos 3 meses:': dtype('O'),
 'Planea usted o algún miembro del hogar adquirir un vehículo en los próximos 3 meses': dtype('O'),
 'Planea usted o algún miembro del hogar adquirir un inmueble (casa, departamento, pieza) en los próximos 3 meses': dtype('O'),
 'Uste

In [672]:
d['Ingreso per cápita'] = round(np.log(d['Ingreso per cápita']), 2)
d['Cuántos años trabaja'] = d['Cuántos años trabaja'].astype(int)

Creamos una lista de variables para hacer dummies

In [673]:
# Resivamos una por una las variables

for i in d.columns:
    print(i)

Area
Sexo
Edad
Seguro Social - Alternativa 1
Seguro Social - Alternativa 2
Estado civil
Nivel de instrucción
Cómo se considera
Categoría de ocupación
Cuántos años trabaja
Condición de actividad
Rama de Actividad CIIU4 (población ocupada de 15 años y más)
Ingreso per cápita
Pobreza
Forma de tenencia de la vivienda
Con relación al mes anterior, la situación económica de su hogar es:
Cómo cree usted que será la situación económica de su hogar dentro de los próximos 3 meses:
Planea usted o algún miembro del hogar adquirir un vehículo en los próximos 3 meses
Planea usted o algún miembro del hogar adquirir un inmueble (casa, departamento, pieza) en los próximos 3 meses
Usted o algún miembro del hogar tiene planes de endeudamiento en los próximos 3 meses (bancos, financieras, etc.)
Con relación al mes anterior, la situación económica del país es:
Usted considera que durante los próximos 3 meses la situación económica del país será:
En los 3 meses anteriores, considera que la situación en Ecua

In [674]:
# Creamos la lista

VAR = ['Area',
       'Sexo',
       'Seguro Social - Alternativa 1',
       'Seguro Social - Alternativa 2',
       'Estado civil',
       'Nivel de instrucción',
       'Cómo se considera',
       'Categoría de ocupación',
       'Condición de actividad',
       'Rama de Actividad CIIU4 (población ocupada de 15 años y más)',
       'Tipo de pobreza',
       'Forma de tenencia de la vivienda',
       'Con relación al mes anterior, la situación económica de su hogar es:',
       'Cómo cree usted que será la situación económica de su hogar dentro de los próximos 3 meses:',
       'Planea usted o algún miembro del hogar adquirir un vehículo en los próximos 3 meses',
       'Planea usted o algún miembro del hogar adquirir un inmueble (casa, departamento, pieza) en los próximos 3 meses',
       'Usted o algún miembro del hogar tiene planes de endeudamiento en los próximos 3 meses (bancos, financieras, etc.)',
       'Con relación al mes anterior, la situación económica del país es:',
       'Usted considera que durante los próximos 3 meses la situación económica del país será:',
       'En los 3 meses anteriores, considera que la situación en Ecuador para encontrar/mejorar un puesto de trabajo es:',
       'Considera usted que la situación en Ecuador para encontrar/mejorar un puesto de trabajo dentro de 3 meses será:',
       'Género del jefe/a de hogar',
       'Tipology']

In [675]:
# Configiguramos la base d con dummies, sin nas

d = pd.get_dummies(d, VAR, dummy_na=False)

## Grupos de entrenamiento y predicción

In [676]:
y = d['Elección política']
y

0       1
1       0
2       0
3       0
4       0
       ..
8498    0
8499    0
8500    0
8501    0
8502    0
Name: Elección política, Length: 8503, dtype: int64

In [680]:
x = d.drop(columns={'Elección política', 'Ingreso per cápita'})
x

,Edad,Cuántos años trabaja,Cónyuge,Hijo o Hija,Parientes,No parientes,Empleado(a) doméstico(a),Número de mujeres en el hogar,Número de personas del hogar,Area_ Rural,...,Género del jefe/a de hogar_Mujer,Tipology_Hogar tipo 1,Tipology_Hogar tipo 10,Tipology_Hogar tipo 11,Tipology_Hogar tipo 14,Tipology_Hogar tipo 16,Tipology_Hogar tipo 4,Tipology_Hogar tipo 5,Tipology_Hogar tipo 8,Tipology_Hogar tipo 9
0,74.0,0,0,0,0,0,0,1,1,1,...,1,1,0,0,0,0,0,0,0,0
1,74.0,0,0,0,0,0,0,1,1,1,...,1,1,0,0,0,0,0,0,0,0
2,74.0,0,0,0,0,0,0,1,1,1,...,1,1,0,0,0,0,0,0,0,0
3,74.0,0,0,0,0,0,0,1,1,1,...,1,1,0,0,0,0,0,0,0,0
4,74.0,0,0,0,0,0,0,1,1,1,...,1,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8498,32.0,0,1,3,0,1,0,3,5,0,...,0,0,1,0,0,0,0,0,0,0
8499,32.0,0,1,3,0,1,0,3,5,0,...,0,0,1,0,0,0,0,0,0,0
8500,32.0,0,1,3,0,1,0,3,5,0,...,0,0,1,0,0,0,0,0,0,0
8501,32.0,0,1,3,0,1,0,3,5,0,...,0,0,1,0,0,0,0,0,0,0


# Model

In [681]:
# importing train_test_split from sklearn

from sklearn.model_selection import train_test_split

# splitting the data

x_train, x_test, y_train, y_test = train_test_split(xa, y, test_size = 0.2, random_state=42)

In [682]:
# importing module

from sklearn.linear_model import LinearRegression

# creating an object of LinearRegression class

LR = LinearRegression()

# fitting the training data

LR.fit(x_train, y_train)

LinearRegression()

In [683]:
y_prediction =  LR.predict(x_test)
y_prediction

array([0.11658994, 0.10245657, 0.1195478 , ..., 0.1768991 , 0.06223995,
       0.13792588])

In [688]:
# importando módulo r2_score

from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

# predecir la puntuación de precisión

score = r2_score(y_test, y_prediction)

print('r2 socor is', score)
print('mean_sqrd_error is ==', mean_squared_error(y_test, y_prediction))
print('root_mean_squared error of is ==', np.sqrt(mean_squared_error(y_test, y_prediction)))

r2 socre is -0.021430244891250005
mean_sqrd_error is == 0.09806701298897731
root_mean_squared error of is == 0.3131565311293656


Lista de variables que no funciona en el modelo:

1. Ingreso per cápita

In [689]:
# Notes
# https://datapeaker.com/big-data/curso-de-ciencia-de-datos-capacitacion-en-ciencia-de-datos/